In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop , Adam
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error , mean_absolute_error , mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import LSTM , Dense , Dropout , GRU , Concatenate , Input , Conv1D , InputLayer
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
import math

In [31]:
#df_btc = pd.read_csv("../../cryptoData/BTC_1h_data.csv")
df_btc = pd.read_csv("BTC_1h_data.csv")

<Strong> Grabbing the closing price (univariate) </Strong>

In [32]:
# Removing everything but the closing price
btc_data = df_btc.values[:, 4 ,].astype(float)

btc_data[-1]

43534.54

<Strong> Scaling the data  </Strong>

In [33]:
percTrain = 70
percVal = 20 

In [34]:
scaler = MinMaxScaler()
    
onePercent = len(btc_data) // 100
numberTraining = onePercent * percTrain

reshaped_data = btc_data.reshape(-1,1)

#Just scaling on training data otherwise it would be leakage
scaler.fit(reshaped_data[:numberTraining])
scaled_btc = scaler.transform(reshaped_data)

<Strong> Creating Matrix in Sliding window form <Strong>

In [35]:
def sliding_window(elements, window_size):
    
    data = [] 
    targets = []
    
    if len(elements) <= window_size:
        return elements
    
    for i in range(len(elements) - window_size ):
        
        data.append(elements[i:i+window_size])
        targets.append(elements[i+window_size])
        
    return np.array(data) , np.array(targets)

In [36]:
# Using 24 datapoints to predict the 25th

window_length = 24
features = 1

sliding_winda_btc = sliding_window(scaled_btc , window_length)

<Strong> Splitting the data into train , val , test </Strong>

In [37]:
# Splitting the data after creating the sliding window data
def splitting_train_test(data):
        
    onePercent = len(data[1]) // 100
    
    numberTraining = onePercent * percTrain
    numberValidation = onePercent * percVal
    
    trainingData = data[0][:numberTraining] , data[1][:numberTraining]
    validationData = data[0][numberTraining : numberTraining + numberValidation] , data[1][numberTraining : numberTraining + numberValidation]
    testData = data[0][numberTraining + numberValidation:] , data[1][numberTraining + numberValidation:] 
    
    #Returning tuples of (sliding-window , target_values)
    return trainingData , validationData , testData

In [38]:
btc_train , btc_val , btc_test = splitting_train_test(sliding_winda_btc)

print(btc_train[0].shape)

(27370, 24, 1)


In [39]:
# Hyper Parameters 

# How many hidden layers we should have 
# Learning rate
# Kernel Size
# Window Size
#Filters

In [40]:

def build_model(hp):
    
    hp_dense_learning_rate = hp.Choice('learning_rate', values = [0.002 , 0.004 , 0.006 , 0.008 , 0.01] )
    hp_num_layers = hp.Int('num_layers', 2 , 10)   
    hp_num_filters = hp.Choice('filters' , values = [32 , 64 , 128])
    
    dilations = 1
    
    model = models.Sequential()

    model.add(Conv1D(filters=hp_num_filters, kernel_size=3, activation='relu', input_shape=(window_length, 1), dilation_rate=dilations , padding = 'causal'))
    
    for i in range (hp_num_layers):
        
        dilations = dilations * 2
        
        model.add(Conv1D(filters=hp.Choice('loop_filter_'+ str(i) , values = [32 , 64 , 128] ), kernel_size=3, activation='relu', dilation_rate=dilations , padding = 'causal'))
    
    model.add(Dense(1))
    
    opt = Adam(learning_rate=hp_dense_learning_rate)
    model.compile(optimizer=opt , loss = 'mse')
    
    return model
    
    

In [41]:
tuner = RandomSearch (
    build_model,
    objective = "val_loss",
    max_trials=50,
    executions_per_trial=1,
    directory = 'tcn',
    project_name='tcn_layers_learningrate'
)

In [42]:
tuner.search(btc_train[0] , btc_train[1] , epochs=300 , validation_data=btc_val , batch_size = 512)
#,  callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=30)]


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.008             |?                 
num_layers        |8                 |?                 
filters           |128               |?                 
loop_filter_0     |128               |?                 
loop_filter_1     |32                |?                 

Epoch 1/300
54/54 [==============================] - 6s 74ms/step - loss: 0.0213 - val_loss: 0.1781
Epoch 2/300
54/54 [==============================] - 3s 55ms/step - loss: 5.8906e-04 - val_loss: 0.1035
Epoch 3/300
54/54 [==============================] - 3s 56ms/step - loss: 3.9164e-04 - val_loss: 0.0180
Epoch 4/300
54/54 [==============================] - 3s 56ms/step - loss: 5.9885e-04 - val_loss: 0.0253
Epoch 5/300
54/54 [==============================] - 3s 56ms/step - loss: 3.0514e-04 - val_loss: 0.0385
Epoch 6/300
54/54 [==============================] - 3s 56ms/step - loss: 3.3028e-04 - val_loss: 0.0369
Epoch 7/30

KeyboardInterrupt: 

In [43]:
# Full Converage = 1 + (kernel_size-1) . dilation_base^n - 1 / dilation_base - 1 >= l
# Number of required layers = log (dilation_base) ( ((input_length - 1) . (dilation_base - 1) / (k - 1) ) + 1 )
# Padding for each layer dilation_base ^ layers below our current . (kernel_size - 1) below = input

# For full coverage we need 4 layers (we are using a base of 2)


model1 = models.Sequential()

# Casual adds padding to the start of input sequence
model.add(Conv1D(filters=4, kernel_size=3, activation='relu', input_shape=(window_length, 1), dilation_rate=1 , padding = 'causal'))

model.add(Conv1D(filters=4, kernel_size=3, activation='relu' , padding = 'causal', dilation_rate=2))

# For full coverage we need 4 layers 
model.add(Conv1D(filters=4, kernel_size=3, activation='relu' , padding = 'causal' , dilation_rate=4))

# For full coverage we need 4 layers 
model.add(Conv1D(filters=4, kernel_size=3, activation='relu' , padding = 'causal', dilation_rate= 8))

model.add(Dense(1))


model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 24, 4)             16        
                                                                 
 conv1d_10 (Conv1D)          (None, 24, 4)             52        
                                                                 
 conv1d_11 (Conv1D)          (None, 24, 4)             52        
                                                                 
 conv1d_12 (Conv1D)          (None, 24, 4)             52        
                                                                 
 dense_1 (Dense)             (None, 24, 1)             5         
                                                                 
Total params: 177
Trainable params: 177
Non-trainable params: 0
_________________________________________________________________


In [44]:
opt = Adam(0.008)
model.compile(optimizer=opt , loss = 'mse')

In [45]:
earlyStopping = EarlyStopping(monitor = 'val_loss' , patience = 30 , mode = 'min' , verbose = 1)

In [46]:
history = model.fit(btc_train[0] , btc_train[1] , validation_data = btc_val  , batch_size = 512  , epochs =300 , verbose = 1 , callbacks=[earlyStopping])

num_epochs = earlyStopping.stopped_epoch

Epoch 1/300
54/54 [==============================] - 1s 9ms/step - loss: 0.0193 - val_loss: 0.2177
Epoch 2/300
54/54 [==============================] - 0s 4ms/step - loss: 0.0023 - val_loss: 0.1091
Epoch 3/300
54/54 [==============================] - 0s 4ms/step - loss: 0.0013 - val_loss: 0.0531
Epoch 4/300
54/54 [==============================] - 0s 4ms/step - loss: 7.7124e-04 - val_loss: 0.0259
Epoch 5/300
54/54 [==============================] - 0s 4ms/step - loss: 5.3104e-04 - val_loss: 0.0167
Epoch 6/300
54/54 [==============================] - 0s 4ms/step - loss: 3.9321e-04 - val_loss: 0.0122
Epoch 7/300
48/54 [=========================>....] - ETA: 0s - loss: 3.2991e-04

KeyboardInterrupt: 